In [1]:
def bellman_ford(start,V, E):
    """
    Find the shortest paths to all vertices given a starting vertex
    Returns array of shortest distances
    If Negative weight cycle is present then returns False"""
    dist = [float("Inf")] * V
    path = [[]*V]
    dist[start] = 0
    for _ in range(V - 1):
            for i in E:
                if dist[i[0]] != float("Inf") and dist[i[0]] + i[2] < dist[i[1]]:
                    dist[i[1]] = dist[i[0]] + i[2]
    for i in E:
        if dist[i[0]] != float("Inf") and dist[i[0]] + i[2] < dist[i[1]]:
            return False    
    return dist


def get_shortest(original):
    from itertools import product

    #create edges for Bellman_ford_algorithm
    l = len(original)
    combos = list(product(list(range(l)),list(range(l))))
    b = [i for j in original for i in j]
    combos = [list(i) for i in combos]
    
    #create shortest paths using bellman_ford
    [i.append(j) for i,j in zip(combos,b)]
    shortest = []
    for i in range(l):
        s = (bellman_ford(i,l,combos))        
        if s == False:
            return False
        shortest.append(s)
    return shortest


def trim_graph(original, shortest):
    """
    Returns a graph that has been 'trimmed' to include only paths that 
    are in fact the shortest between adjacent nodes
    """
    l = len(original)
    trimmed = [[float("inf") for i in range(l)] for j in range(l)]
    for i in range(len(trimmed)):
        for j in range(len(trimmed)):
            if i == j:
                continue
            if original[i][j] == shortest[i][j]:
                trimmed[i][j] = original[i][j]
    return trimmed

def make_edge_dict(trimmed):
    """
    Make an edges dictionary from the trimmed graph, these are the allowable paths of shortest distance
    """
    edges = {}
    for i in range(len(trimmed)):
        edges[i] = [index for index,j in enumerate(trimmed[i]) if j != float("inf")]
    return edges

def get_best(solutions):
    """
    Return the solution that is the longest and contains the smallest ID's
    """
    maximum = max([len(i) for i in solutions])
    equal_solutions = [i for i in solutions if len(i) == maximum]
    [i.sort() for i in equal_solutions]
    equal_solutions.sort()
    best_solution = [i-1 for i in equal_solutions[0]]
    return best_solution

def solution(original, time_left):
    #first we check for negative loops while also making a shortest distance array
    s = get_shortest(original)
    if s != False:
        t = trim_graph(original,get_shortest(original))
    else:
        return [i for i in range(len(original)-2)]
    d = make_edge_dict(t)

    solutions = []
    solutions.append([])

    try:
        solutions = find_bunnies(vertex=0, 
                        room_state = {key:[] for key in d.keys()},
                        bulkhead=len(original)-1, 
                        saved=[], 
                        dictionary=d, 
                        shortest=t,
                        time=time_left, 
                        solutions=solutions)
    except ValueError:
        return [i for i in range(len(original)-2)]
    
    best_solution = get_best(solutions)
    return best_solution
    
def find_bunnies(vertex, room_state, bulkhead, saved, dictionary, shortest, time, solutions):
    """
    Searches for bunnies recursively using a depth first search and some basic heuristics
    Parameters: Vertex (Room) you are currently in, room_state is a dictionary of states you've had while entering a room,
    bulkhead is the index of the final room, saved is a list of the bunnies you've saved, dictionary contains the edges that
    are truly shortest paths, shortest is an array that contains the values to help you calculate time and 
    solutions is a list of all solutions found.
    Returns solutions"""

    #Have I been in this room before in the same state I'm currently in?
    if saved in room_state[vertex]:
        return

    #Is there no way to return to home base without running out of time?
    if time < 0 and shortest[vertex][bulkhead]>0:
        return

    #Save the bunny at our current location
    if vertex != 0 and vertex != bulkhead and vertex not in saved:
        saved = list(saved+[vertex])

    #the error means we have found the best solution so we unwind the stack
    if vertex == bulkhead:
        if time < 0:
            return            
        if saved not in solutions:
            solutions.append(list(saved))
        if len(saved) == len(shortest)-2:
            raise  ValueError

    #visit each room in the queue
    to_visit = list(dictionary[vertex])
    for i in to_visit:
        room_state[vertex].append(list(saved))
        find_bunnies(vertex=i, 
                     room_state=room_state, 
                     bulkhead=bulkhead, 
                     saved=saved, 
                     dictionary=dictionary, 
                     shortest=shortest, 
                     time=time-shortest[vertex][i], 
                     solutions=solutions)

    return solutions

In [2]:
case1 = [[0, 1, 1, 1, 1],
         [1, 0, 1, 1, 1],
         [1, 1, 0, 1, 1],
         [1, 1, 1, 0, 1],
         [1, 1, 1, 1, 0]]
#Time limit: 3
#Expected: [0, 1]

case2 = [[0, 2, 2, 2, -1],
         [9, 0, 2, 2, -1],
         [9, 3, 0, 2, -1],
         [9, 3, 2, 0, -1],
         [9, 3, 2, 2, 0]]
#Time limit: 1
#Expected: [1, 2]

case3 = [[0, 2, 2, 2, -1],
         [9, 0, 2, 2, 0],
         [9, 3, 0, 2, 0],
         [9, 3, 2, 0, 0],
         [-1, 3, 2, 2, 0]]
#Time limit: -500
#Expected: [0, 1, 2]

case4 = [[1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1]]
#Time limit: 1
#Expected: []

case5 = [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]]
#Time limit: 2
#Expected: [0]

case6 = [[0, 5, 11, 11, 1],
         [10, 0, 1, 5, 1],
         [10, 1, 0, 4, 0],
         [10, 1, 5, 0, 1],
         [10, 10, 10, 10, 0]]
#Time limit: 10
#Expected: [0, 1]

case7 = [[0, 10, 10, 10, 1],
         [0, 0, 10, 10, 10],
         [0, 10, 0, 10, 10],
         [0, 10, 10, 0, 10],
         [1, 1, 1, 1, 0]]
#Time limit: 5
#Expected: [0, 1]

case8 = [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]
#Time limit: 1
#Expected: [0, 1, 2]


case9 = [[2, 2],
         [2, 2]]
#Time limit: 1
#Expected: []

case10 = [[0, 10, 10, 1, 10],
          [10, 0, 10, 10, 1],
          [10, 1, 0, 10, 10],
          [10, 10, 1, 0, 10],
          [1, 10, 10, 10, 0]]
#Time limit: 6
#Expected: [0,1,2]
time_limits = [3,1,-500,1,2,10,5,1,1,6]
expected_results = [[0,1],
                    [1,2],
                    [0,1,2],
                    [],
                    [0],
                    [0,1],
                    [0,1],
                    [0,1,2],
                    [],
                    [0,1,2]]

In [3]:
test_dictionary = {}
test_dictionary["case1"] = case1
test_dictionary["case2"] = case2
test_dictionary["case3"] = case3
test_dictionary["case4"] = case4
test_dictionary["case5"] = case5
test_dictionary["case6"] = case6
test_dictionary["case7"] = case7
test_dictionary["case8"] = case8
test_dictionary["case9"] = case9
test_dictionary["case9"] = case9
test_dictionary["case10"] = case10

In [4]:
for num,time,result in zip(range(1,11),time_limits,expected_results):
    s = solution(test_dictionary[f'case{num}'],time)
    if s == result:
        print(f'case {num} passed')
    else:
        print(f'case {num} failed')
        print(f'Actual: {s} Expected: {result}')

case 1 passed
case 2 passed
case 3 passed
case 4 passed
case 5 passed
case 6 passed
case 7 passed
case 8 passed
case 9 passed
case 10 passed


In [5]:
trim_graph(case7,get_shortest(case7))

[[inf, inf, inf, inf, 1],
 [0, inf, inf, inf, inf],
 [0, inf, inf, inf, inf],
 [0, inf, inf, inf, inf],
 [1, 1, 1, 1, inf]]